In [1]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import os
from tqdm import tqdm
import cv2

In [2]:
# from: https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_abi_lat_lon.php
def calculate_degrees(file_id):
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]  # E/W scanning angle in radians
    y_coordinate_1d = file_id.variables['y'][:]  # N/S elevation angle in radians
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    # Ignore numpy errors for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
    np.seterr(all='ignore')
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return abi_lat, abi_lon


In [3]:
pr_lon = [15, 20]
pr_lat = [-70, -63]

In [4]:
def get_pr_bounds(nc, vis=False):
    long, lat = calculate_degrees(nc)
    print(np.min(lat), np.max(lat))
    long = np.array(long)
    lat = np.array(lat)

    lat_ind = np.where((lat > pr_lat[0]) & (lat < pr_lat[1]), 1, 0)
    lat_bin = np.zeros_like(lat)
    lat_bin[lat_ind == 1] = 1

    long_ind = np.where((long > pr_lon[0]) & (long < pr_lon[1]), 1, 0)
    long_bin = np.zeros_like(long)
    long_bin[long_ind == 1] = 1 
    
    pr_bin = np.zeros_like(long)
    pr_bin[(lat_ind == 1) & (long_ind == 1)] = 1

    if vis:
        #visualize data
        plt.imshow(long, cmap='viridis')  # Adjust the colormap ('viridis', 'gray', 'jet', etc.) as needed
        plt.title('Longitude')  # Set a title for the plot
        plt.xlabel('X Image')
        plt.ylabel('Y Image')
        plt.show()

        plt.imshow(pr_bin, cmap='viridis')  # Adjust the colormap ('viridis', 'gray', 'jet', etc.) as needed
        plt.title('PR')  # Set a title for the plot
        plt.xlabel('X Image')
        plt.ylabel('Y Image')
        plt.show()

    min_y, max_y = np.where(pr_bin == 1)[0][0], np.where(pr_bin == 1)[0][-1]
    min_x, max_x = np.where(pr_bin == 1)[1][0], np.where(pr_bin == 1)[1][-1]
    return min_x, max_x, min_y, max_y


def get_image(file_path, bounds):
    nc = netCDF4.Dataset(file_path)

    #print cloud pixel data
    # print(nc.variables['geospatial_lat_lon_extent'])
    # print(nc.variables.keys())
    # #get longitude and latitude data for each pixel
    # print(nc.variables['x'] * nc.variables['scale_factor_x'] + nc.variables['offset_x'])
    # print(nc.variables['y'])
    data = np.array(nc.variables['Cloud_Probabilities'])

    if bounds:
        min_x, max_x, min_y, max_y = bounds
        data = data[min_y:min_y+255, min_x+100:min_x+355]
    nc.close()


    return data

def show_data(img):
    plt.imshow(img, cmap='viridis')  # Adjust the colormap ('viridis', 'gray', 'jet', etc.) as needed
    plt.title('Daytime Cloud Pixels')  # Set a title for the plot
    plt.xlabel('X Image')
    plt.ylabel('Y Image')
    plt.show()

def create_mp4(imgs, filename):
    v = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), fps=10, frameSize=(len(imgs[0][0]), len(imgs[0])), isColor=False)
    for i in tqdm(range(len(imgs))):
        v.write(imgs[i])#(imgs[i] / (2**8)).astype(np.uint8)
    v.release()

# get_long_lat('2023/001/00/OR_ABI-L2-ACMC-M6_G16_s20230010001173_e20230010003546_c20230010005038.nc')

In [5]:
data_folder = '2023/'

#walk through the directory and get the file name
file_paths = []
sub_folders = [f.path for f in os.scandir(data_folder) if f.is_dir()]
sub_sub_folders = []
for folder in tqdm(sub_folders):
    new_folders = [f.path for f in os.scandir(folder) if f.is_dir()]
    sub_sub_folders.extend(new_folders)
    
for folder in tqdm(sub_sub_folders):
    for f in os.scandir(folder):
        if f.is_file() and f.name.endswith('.nc'):
            file_paths.append(f.path)

file_paths.sort()

bounds = get_pr_bounds(netCDF4.Dataset(file_paths[0]), vis=False)
print(bounds)
imgs = []
for path in tqdm(file_paths):
    
    imgs.append(get_image(path, bounds))
    # imgs.append(np.ones((255, 255)))

# Normalize values between 0-255
imgs = np.array(imgs)
imgs = imgs / 256  
imgs = np.array(imgs, dtype=np.uint8)

imgs = [imgs[i:i+20] for i in range(0, len(imgs), 20)]
imgs = imgs[:-1]
print(len(imgs))
print(imgs[0].shape)

100%|██████████| 1200/1200 [00:00<00:00, 2104.26it/s]
C:\Users\Sam\AppData\Local\Temp\ipykernel_31196\556323288.py:20: RuntimeWarning: invalid value encountered in sqrt
  r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)


-152.05498 -52.947044
(2067, 2441, 1222, 1479)


100%|██████████| 14398/14398 [00:03<00:00, 4553.69it/s]


719
(20, 255, 255)


In [6]:
def create_npy(imgs, filename):
    np.save(filename, imgs)

In [7]:
# create_mp4(imgs, '2023-full.mp4')

In [8]:
create_npy(imgs, '2023-full.npy')

In [9]:
imgs_npy = np.load('2023-full.npy')